In [1]:
from neo4j import GraphDatabase

In [2]:
class Neo4jConnection:
    def __init__(self, uri, user, password):
        self.driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        if self.driver is not None:
            self.driver.close()

# Метод, который передает запрос в БД
    def query(self, query, db=None):
        assert self.driver is not None, "Driver not initialized!"
        session = None
        response = None
        try:
            session = self.driver.session(database=db) if db is not None else self.driver.session()
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally:
            if session is not None:
                session.close()
        return response

In [3]:
conn = Neo4jConnection(uri="bolt://localhost:7687", user="admin", password="00000000")

In [10]:
conn.query("CREATE OR REPLACE DATABASE social")

[]

In [9]:
query_string = '''
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/UnicornTowa/neo4j-demo/main/persons.csv'
AS line FIELDTERMINATOR ','
MERGE (person:Person {PersonID: line.PersonID})
  ON CREATE SET person.PersonName = line.PersonName, 
                person.PersonResidence = line.PersonResidence, 
                person.PersonSex = line.PersonSex, 
                person.PersonAge = toInteger(line.PersonAge);
'''
conn.query(query_string, db='social')

[]

In [11]:
query_string = '''
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/UnicornTowa/neo4j-demo/main/friends.csv'
AS line FIELDTERMINATOR ','
MATCH (p1:Person {PersonID:line.Person1})
MATCH (p2:Person {PersonID:line.Person2})
MERGE (p1)-[:KNOWS {Type:line.type}]-(p2)
'''
conn.query(query_string, db='social')

[]

In [12]:
query_string = '''
LOAD CSV WITH HEADERS FROM
'https://raw.githubusercontent.com/UnicornTowa/neo4j-demo/main/relationships.csv'
AS line FIELDTERMINATOR ','
MATCH (p1:Person {PersonID:line.Person1})
MATCH (p2:Person {PersonID:line.Person2})
MERGE (p1)-[:LOVES]-(p2)
'''
conn.query(query_string, db='social')

[]